## 문장의 단어를 하나씩 가져오기
for 반복문에 in 키워드를 이용하면 리스트의 원소를 하나씩 가져와 변수에 저장할 수 있습니다.

여기에 리스트의 길이를 구하는 len()와, 연속된 정수를 만들어주는 range() 함수를 함께 사용하면 원소의 인덱스를 가져올 수 있습니다.

### elice-utils.py

In [2]:
# elice-utils
# maintainer: Suin Kim (suin@elicer.com) and Jungkook Park (jk@elicer.com)

import base64
import mimetypes
import os
import urllib.parse
import urllib.request


class EliceUtils(object):

    def __init__(self):
        self._execution_token = os.getenv('EXECUTION_TOKEN')
        self._executor_ip = os.getenv('EXECUTOR_IP')
        self._executor_com_port = os.getenv('EXECUTOR_COM_PORT')
        self._otp_key = None

        self._local_mode = False

        if not all((self._execution_token, self._executor_ip, self._executor_com_port)):
            self._local_mode = True

            print('=== NON-ELICE ENVIRONMENT ===')
            print('Warning: This script is running on the non-elice environment. '
                  'All outputs will be redirected to standard output.')
            print('=============================')

    def _send(self, url, data):
        if self._local_mode:
            msg_type = data['type']
            msg_data = data['data']

            if msg_type in ['grader', 'score']:
                print('[%s] %s' % (msg_type, msg_data), end='')
            else:
                print('[%s]' % msg_type, end='')

            return

        data_encoded = urllib.parse.urlencode(data)
        q = urllib.request.Request(url,
                                   data=data_encoded.encode('utf-8'))

        try:
            urllib.request.urlopen(q)
        except Exception:
            raise Exception('Failed to send message to elice.')

    def _handle_image(self, filepath):
        mtype, _ = mimetypes.guess_type(filepath)

        if mtype is None or not mtype.startswith('image/'):
            raise ValueError('Invalid image filepath.')

        with open(filepath, 'rb') as f:
            data = 'data:%s;base64,%s' % (
                mtype,
                base64.b64encode(f.read()).decode('utf-8')
            )

        return data

    def _handle_file(self, filepath):
        mtype, _ = mimetypes.guess_type(filepath)

        with open(filepath, 'rb') as f:
            data = '%s;data:%s;base64,%s' % (
                os.path.basename(filepath),
                mtype or 'application/octet-stream',
                base64.b64encode(f.read()).decode('utf-8')
            )

        return data

    def send(self, msg_type, msg_data):
        self._send(
            'http://%s:%s/comm/send/%s' % (self._executor_ip,
                                           self._executor_com_port,
                                           self._execution_token),
            {'type': msg_type, 'data': msg_data}
        )

    def send_image(self, filepath):
        self.send('image', self._handle_image(filepath))

    def send_file(self, filepath):
        self.send('file', self._handle_file(filepath))

    def secure_init(self):
        if self._local_mode:
            return

        try:
            r = urllib.request.urlopen(
                'http://%s:%s/comm/secure/init/%s' % (self._executor_ip,
                                                      self._executor_com_port,
                                                      self._execution_token)
            )
        except Exception:
            raise Exception('Failed to initialize elice util secure channel.')

        self._otp_key = r.read().decode('utf-8')

    def secure_send(self, msg_type, msg_data):
        self._send(
            'http://%s:%s/comm/secure/send/%s/%s' % (self._executor_ip,
                                                     self._executor_com_port,
                                                     self._execution_token,
                                                     self._otp_key),
            {'type': msg_type, 'data': msg_data}
        )

    def secure_send_image(self, filepath):
        self.secure_send('image', self._handle_image(filepath))

    def secure_send_file(self, filepath):
        self.secure_send('file', self._handle_file(filepath))

    def secure_send_grader(self, msg):
        self.secure_send('grader', msg)

    def secure_send_score(self, score):
        self.secure_send('score', score)


### main.py

In [3]:
# 트럼프 대통령의 1월 1~3일 트윗을 각각 리스트의 원소로 저장합니다.
trump_tweets = [
    'Will be leaving Florida for Washington (D.C.) today at 4:00 P.M. Much work to be done, but it will be a great New Year!',
    'Companies are giving big bonuses to their workers because of the Tax Cut Bill. Really great!',
    'MAKE AMERICA GREAT AGAIN!'
]

def date_tweet(tweet):
    # index에 0~2을 차례대로 저장하여 반복문을 실행합니다.
    for index in range(len(tweet)):
        print('2018년 1월 ' + str(index+1) + '일: ' + tweet[index])


# 실행 결과를 확인하기 위한 코드입니다.
date_tweet(trump_tweets)

2018년 1월 1일: Will be leaving Florida for Washington (D.C.) today at 4:00 P.M. Much work to be done, but it will be a great New Year!
2018년 1월 2일: Companies are giving big bonuses to their workers because of the Tax Cut Bill. Really great!
2018년 1월 3일: MAKE AMERICA GREAT AGAIN!


## 단어의 일부분 가져오기
인덱스를 이용하면 문자열 또는 리스트의 특정 요소에 접근할 수 있습니다. 인덱스는 0부터 시작하며 -1은 맨 마지막 문자 또는 요소를 가리킵니다.

시작 인덱스와 끝 인덱스를 이용하면 특정 구간의 요소를 리스트형으로 접근할 수 있습니다. 끝 인덱스를 생략하면 시작 인덱스부터 마지막 요소까지 접근합니다.



In [4]:
# 트럼프 대통령 트윗을 공백 기준으로 분리한 리스트입니다. 수정하지 마세요.
trump_tweets = ['thank', 'you', 'to', 'president', 'moon', 'of', 'south', 'korea', 'for', 'the', 'beautiful', 'welcoming', 'ceremony', 'it', 'will', 'always', 'be', 'remembered']

def print_korea(text):
    # 아래 코드를 작성하세요.
    for word in text:
      if word[0] == "k":
        print(word)
    
    
# 아래 주석을 해제하고 결과를 확인해보세요.  
print_korea(trump_tweets)

korea


## 단어의 첫 글자 확인하기
startswith() 메소드를 이용하면 단어가 특정 문자열로 시작하는지 쉽게 확인할 수 있습니다.

In [6]:
# 트럼프 대통령 트윗을 공백 기준으로 분리한 리스트입니다. 수정하지 마세요.
trump_tweets = ['thank', 'you', 'to', 'president', 'moon', 'of', 'south', 'korea', 'for', 'the', 'beautiful', 'welcoming', 'ceremony', 'it', 'will', 'always', 'be', 'remembered']

def print_korea(tweet):
    '''
    문자열로 구성된 리스트에서 k로 시작하는 문자열을 출력합니다.
    '''
    for text in tweet:
        if text.startswith("k"):
            print(text)
    
    
    
# 아래 주석을 해제하고 결과를 확인해보세요.  
print_korea(trump_tweets)

korea


## 문장을 단어 단위로 구분하기
split() 메소드는 특정 문자를 기준으로 문자열을 분리합니다. 입력값을 넣지 않을 경우 공백을 기준으로 분리합니다. 분리된 문자열은 리스트의 원소로 저장됩니다.

In [7]:
# 트럼프 대통령의 트윗으로 구성된 문자열입니다. 수정하지 마세요. 
trump_tweets = "thank you to president moon of south korea for the beautiful welcoming ceremony it will always be remembered"

def break_into_words(text):
    # 아래 코드를 작성하세요.
    words = text.split()
    
    return words


# 아래 주석을 해제하고 결과를 확인해보세요.  
print(break_into_words(trump_tweets))

['thank', 'you', 'to', 'president', 'moon', 'of', 'south', 'korea', 'for', 'the', 'beautiful', 'welcoming', 'ceremony', 'it', 'will', 'always', 'be', 'remembered']


## 새로운 단어 추가하기
append()는 리스트를 다룰 때 사용되는 가장 기본적인 메소드로, 리스트의 맨 마지막에 새로운 요소를 추가합니다.

In [9]:
# 트럼프 대통령 트윗을 공백 기준으로 분리한 리스트입니다. 수정하지 마세요.
trump_tweets = ['america', 'is', 'back', 'and', 'we', 'are', 'coming', 'back', 'bigger', 'and', 'better', 'and', 'stronger', 'than', 'ever', 'before']

def make_new_list(text):
    # 아래 코드를 작성하세요.
    new_list = []
    for word in text:
      if word.startswith("b"):
        new_list.append(word)
  
    return new_list


# 아래 주석을 해제하고 결과를 확인해보세요.  
new_list = make_new_list(trump_tweets)
print(new_list)

['back', 'back', 'bigger', 'better', 'before']


## 대소문자 변환하기
lower(), upper() 메소드를 이용하면 문자열을 쉽게 소문자 또는 대문자로 변환할 수 있습니다.

In [11]:
# 트럼프 대통령의 트윗 세개로 구성된 리스트입니다. 수정하지 마세요.
trump_tweets = [
    "FAKE NEWS - A TOTAL POLITICAL WITCH HUNT!",
    "Any negative polls are fake news, just like the CNN, ABC, NBC polls in the election.",
    "The Fake News media is officially out of control.",
]
 
def lowercase_all_characters(text):
    processed_text = []
    # 아래 코드를 작성하세요.
    
    for sentence in text:
      processed_text.append(sentence.lower())
    
    return processed_text


# 아래 주석을 해제하고 결과를 확인해보세요.  
print('\n'.join(lowercase_all_characters(trump_tweets)))

fake news - a total political witch hunt!
any negative polls are fake news, just like the cnn, abc, nbc polls in the election.
the fake news media is officially out of control.


## 특수기호 삭제하기
replace() 메소드는 문자열에서 특정 문자나 문자열을 다른 문자(열)로 변경할 때 사용됩니다.

replace()는 변경하고 싶은 문자열을 첫번째 입력값으로, 대체할 문자열을 두번째 입력값으로 받습니다.

In [12]:
# 트럼프 대통령의 트윗 세개로 구성된 리스트입니다. 수정하지 마세요.
trump_tweets = [
    "i hope everyone is having a great christmas, then tomorrow it’s back to work in order to make america great again.",
    "7 of 10 americans prefer 'merry christmas' over 'happy holidays'.",
    "merry christmas!!!",
]

def remove_special_characters(text):
    processed_text = []
    # 아래 코드를 작성하세요.
    for sentence in text:
      sentence = sentence.replace(",", "").replace("'", "").replace("!", "")
      processed_text.append(sentence)
    

    return processed_text


# 아래 주석을 해제하고 결과를 확인해보세요.  
print('\n'.join(remove_special_characters(trump_tweets)))

i hope everyone is having a great christmas then tomorrow it’s back to work in order to make america great again.
7 of 10 americans prefer merry christmas over happy holidays.
merry christmas
